In [1]:
import sys

In [2]:
from pytket import Circuit
from tket.circuit import Tk2Circuit
# from tket.protocol import CircuitReplacer

In [3]:
sys.path.append("../../../tket2-rewriting/tket2_rewriting")
from circuit_generation.gadget import CXGadgetBenchmarkCircuit,SimpleLabelFriendlyCircuit
from pytket.circuit.display import render_circuit_jupyter
import numpy as np
from pytket.passes import DecomposeBoxes, AutoRebase
from pytket import OpType

# # Original circuit to be compiled
orig_circuit = SimpleLabelFriendlyCircuit(width=5, depth=10, rng=np.random.default_rng(0))
orig_circuit.n_2qb_gates()

20

In [4]:
# orig_circuit = Circuit(2,2).CX(0,1).X(0).CX(0,1)

In [5]:
orig_circuit

[CX q[0], q[4]; Rz(1.42351) q[1]; Rz(2.65582) q[2]; Rz(0.257444) q[0]; Rz(1.23166) q[1]; Rx(0.780895) q[2]; Rx(1.10993) q[0]; Rz(0.570218) q[1]; CX q[0], q[4]; Rz(3.89692) q[1]; CX q[0], q[3]; Rz(0.614896) q[1]; Rz(0.533245) q[4]; Rz(1.38723) q[0]; Rx(2.11121) q[1]; Rx(0.58456) q[0]; CX q[0], q[3]; CX q[0], q[2]; Rz(2.26565) q[0]; CX q[0], q[2]; CX q[0], q[3]; Rz(1.50711) q[2]; Rz(3.05259) q[0]; Rx(2.11189) q[2]; CX q[0], q[3]; CX q[0], q[4]; Rz(2.82957) q[3]; Rx(3.9156) q[0]; Rz(0.222268) q[3]; CX q[0], q[4]; Rx(1.91947) q[3]; CX q[0], q[2]; Rz(1.46004) q[3]; Rz(0.662846) q[4]; Rz(0.326859) q[0]; Rx(0.518991) q[3]; CX q[0], q[2]; CX q[0], q[4]; Rx(0.580537) q[2]; Rz(1.99787) q[0]; Rz(1.84975) q[2]; CX q[0], q[4]; Rx(0.599331) q[2]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[1]; Rz(0.762347) q[4]; Rx(3.38622) q[0]; CX q[0], q[1]; ]

In [6]:
sys.path.append("../../../tket2-rewriting/tket2_rewriting")

from cx_gadget.cx_gadget import CXGadget
from zzphase_flip.zzphase_flip import ZZPhaseFlip
from three_qubit_squash.three_qubit_squash_gadget import Squash3Gadget
from KAK.kak_gadget import KAKGadget
from label.label_gadget import LabelGadget

In [7]:
circuit_tk2 = Tk2Circuit(orig_circuit)
circuit_tk2

## Existing gadgets

In [8]:
from tket.optimiser import BadgerOptimiser
from time import time
from pytket.passes import GreedyPauliSimp, FullPeepholeOptimise

circuit = orig_circuit.copy()

opt = BadgerOptimiser(
    [
        # ZZPhaseFlip(),
        # CXGadget(),
        # Squash3Gadget(),
        # KAKGadget()
        LabelGadget()
    ]
)

In [9]:
# Optimise circuit and time the optimisation - this is doing something
start_time = time()
opt_circ = opt.optimise(circuit_tk2)
print(f"time taken: {time() - start_time}")


[DEBUG] get_map PID=39219 TID=8688410944 label_map_id=5206770560 map={}
[DEBUG] init_label_map PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: '0', 2: '0', 3: '0', 4: '0'}
[DEBUG] get_label q=0 -> 0 PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: '0', 2: '0', 3: '0', 4: '0'}
[DEBUG] get_label q=4 -> 0 PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: '0', 2: '0', 3: '0', 4: '0'}
[DEBUG]: I recognise a CX
[DEBUG] get_map PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: '0', 2: '0', 3: '0', 4: '0'}
[DEBUG] get_label q=1 -> 0 PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: '0', 2: '0', 3: '0', 4: '0'}
[DEBUG] get_map PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: '0', 2: '0', 3: '0', 4: '0'}
[DEBUG] get_label q=2 -> 0 PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: '0', 2: '0', 3: '0', 4: '0'}
[DEBUG] get_map PID=39219 TID=8688410944 label_map_id=5189981632 map={0: '0', 1: 

1 qubit gate, if CX : 0->1, 1->0, if I: 0->0, 1->1, otherwise ?
2 qubit gate, if CX: control 0 -> target stays as is, control 1 -> target swaps 0->1 or 1->0, control label is unchanged
else: control 0 -> target stays as is, control 1 -> target ?, control label is unchanged

In [10]:
opt_circ1 = opt_circ.to_tket1()
type(opt_circ1) , type(circuit)

(pytket._tket.circuit.Circuit, pytket._tket.circuit.Circuit)

In [11]:
print(f"Two qubit gates: {opt_circ1.n_2qb_gates()} vs previous: {circuit.n_2qb_gates()}")

Two qubit gates: 19 vs previous: 20


In [12]:
orig_circuit

[CX q[0], q[4]; Rz(1.42351) q[1]; Rz(2.65582) q[2]; Rz(0.257444) q[0]; Rz(1.23166) q[1]; Rx(0.780895) q[2]; Rx(1.10993) q[0]; Rz(0.570218) q[1]; CX q[0], q[4]; Rz(3.89692) q[1]; CX q[0], q[3]; Rz(0.614896) q[1]; Rz(0.533245) q[4]; Rz(1.38723) q[0]; Rx(2.11121) q[1]; Rx(0.58456) q[0]; CX q[0], q[3]; CX q[0], q[2]; Rz(2.26565) q[0]; CX q[0], q[2]; CX q[0], q[3]; Rz(1.50711) q[2]; Rz(3.05259) q[0]; Rx(2.11189) q[2]; CX q[0], q[3]; CX q[0], q[4]; Rz(2.82957) q[3]; Rx(3.9156) q[0]; Rz(0.222268) q[3]; CX q[0], q[4]; Rx(1.91947) q[3]; CX q[0], q[2]; Rz(1.46004) q[3]; Rz(0.662846) q[4]; Rz(0.326859) q[0]; Rx(0.518991) q[3]; CX q[0], q[2]; CX q[0], q[4]; Rx(0.580537) q[2]; Rz(1.99787) q[0]; Rz(1.84975) q[2]; CX q[0], q[4]; Rx(0.599331) q[2]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[1]; Rz(0.762347) q[4]; Rx(3.38622) q[0]; CX q[0], q[1]; ]

In [13]:
opt_circ1

[Rz(0.257444) q[0]; Rz(1.42351) q[1]; Rz(2.65582) q[2]; Rx(1.10993) q[0]; Rz(1.23166) q[1]; Rx(0.780895) q[2]; CX q[0], q[4]; Rz(0.570218) q[1]; CX q[0], q[3]; Rz(3.89692) q[1]; Rz(0.533245) q[4]; Rz(1.38723) q[0]; Rz(0.614896) q[1]; Rx(0.58456) q[0]; Rx(2.11121) q[1]; CX q[0], q[3]; CX q[0], q[2]; Rz(2.26565) q[0]; CX q[0], q[2]; CX q[0], q[3]; Rz(1.50711) q[2]; Rz(3.05259) q[0]; Rx(2.11189) q[2]; CX q[0], q[3]; CX q[0], q[4]; Rz(2.82957) q[3]; Rx(3.9156) q[0]; Rz(0.222268) q[3]; CX q[0], q[4]; Rx(1.91947) q[3]; CX q[0], q[2]; Rz(1.46004) q[3]; Rz(0.662846) q[4]; Rz(0.326859) q[0]; Rx(0.518991) q[3]; CX q[0], q[2]; CX q[0], q[4]; Rx(0.580537) q[2]; Rz(1.99787) q[0]; Rz(1.84975) q[2]; CX q[0], q[4]; Rx(0.599331) q[2]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[1]; Rz(0.762347) q[4]; Rx(3.38622) q[0]; CX q[0], q[1]; ]

In [14]:
# Follow with FullPeepholeOptimise
FullPeepholeOptimise().apply(opt_circ1)
print(f"Two qubit gates: {opt_circ1.n_2qb_gates()}")

Two qubit gates: 7


In [15]:
opt_circ1

[TK1(1.38723, 1.10993, 0.257444) q[0]; TK1(0, 0.111207, 1.7372) q[1]; TK1(0.877019, 3.74199, 0.149414) q[2]; CX q[0], q[4]; CX q[0], q[3]; TK1(0, 0, 0.533245) q[4]; TK1(1.31825, 0.58456, 0) q[0]; CX q[0], q[3]; CX q[0], q[4]; TK1(3.08021, 3.47163, 1.51787) q[3]; TK1(0.324728, 1.9156, 0) q[0]; CX q[0], q[4]; CX q[0], q[1]; TK1(0, 0, 1.42519) q[4]; TK1(0, 1.38622, 0) q[0]; CX q[0], q[1]; ]

In [16]:
# With GreedyPauliSimp the result is not as good
circuit = orig_circuit.copy()
GreedyPauliSimp().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 8


In [17]:
opt_circ1

[TK1(1.38723, 1.10993, 0.257444) q[0]; TK1(0, 0.111207, 1.7372) q[1]; TK1(0.877019, 3.74199, 0.149414) q[2]; CX q[0], q[4]; CX q[0], q[3]; TK1(0, 0, 0.533245) q[4]; TK1(1.31825, 0.58456, 0) q[0]; CX q[0], q[3]; CX q[0], q[4]; TK1(3.08021, 3.47163, 1.51787) q[3]; TK1(0.324728, 1.9156, 0) q[0]; CX q[0], q[4]; CX q[0], q[1]; TK1(0, 0, 1.42519) q[4]; TK1(0, 1.38622, 0) q[0]; CX q[0], q[1]; ]

In [18]:
# With FullPeephole the result is not as good
circuit = orig_circuit.copy()
FullPeepholeOptimise().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 8


In [19]:
# With both the result is not as good
circuit = orig_circuit.copy()
GreedyPauliSimp().apply(circuit)
FullPeepholeOptimise().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 8
